# **Projeto : Classificando a vida de fetos**

O objetivo desse projeto é, dado uma tabela com atributos relacionados a saúde de um feto, seria possível prever com mais de 90% de acerto se o feto está com problemas de saúde?

Para começar a EDA, primeiro vamos expor todos os atributos contidos na tabela que foi dada:

* **baseline value(Frequência cardíaca fetal basal)** - mostra a quantidade de batidas do coração por minuto

* **accelerations** - número de acelerações por segundo 

* **fetal_movement** - número de movimento fetal por segundo

* **uterine_contractions** - número de contrações uterinas por segundo

* **light_decelerations** - número de desacelerações leves por segundo

* **severe_decelerations** - número de desacelerações severas por segundo

* **prolongued_decelerations** - número de desacelerações prolongadas por segundo

* **abnormal_short_term_variability** - Porcentagem de tempo com variabilidade anormal de curto prazo

* **mean_value_of_short_term_variability** - Valor médio da variabilidade de curto prazo

* **percentage_of_time_with_abnormal_long_term_variability** - Porcentagem de tempo com variabilidade anormal de longo prazo

* **mean_value_of_long_term_variability** - Valor médio da variabilidade de curto prazo

* **histogram_width** - Largura do histograma feito usando todos os valores de um registro

* **histogram_min** - Valor mínimo do histograma

* **histogram_max** - Valor máximo do histograma

* **histogram_number_of_peaks** - Número de picos no histograma do exame

* **histogram_number_of_zeroes** - Número de zeros no histograma do exame
 
* **histogram_mode** - moda do histograma

* **histogram_mean** - média do histograma

* **histogram_median** - mediana do histograma

* **histogram_variance** - variancia do histograma

* **histogram_tendency** - Tendência do histograma

* **fetal_health(Vida fetal)** - 1: Normal, 2: Suspeita, 3: Patológica

A titulo de coveniencia vou pegar e analisar os atributos que dizem respeito a saúde do feto, ou seja os atributos que falam sobre o histograma serão colocados de lado.

In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score , average_precision_score

df = pd.read_csv('../input/fetal-health-classification/fetal_health.csv')



df.info()


In [ ]:
df.describe()

Como queremos classificar a situação do feto, plotaremos um gráfico para mostrar a quantidade dividida em cada uma classe

In [ ]:
px.bar(df['fetal_health'].value_counts().sort_index(),
       color =df['fetal_health'].value_counts().sort_index().values,
       color_continuous_scale= 'RdBu',
      title = 'Quantidade por classe',
       labels = {'index' :'classe','value':'quantidade','color':'quantidade'}
      )


Observando o gráfico acima, vemos que existe mais fetos saudáveis do que com algum problema. Quando estiver na parte de divisão de dados precisará dividi-los de forma estratificada para evitar viéses. 

Na análise exploratória abaixo, é possivel observar que existem anomalias nos dados, porém optei por não trata-los para aumentar a capacidade de generalização dos algoritmos de machine learning.

In [ ]:
px.histogram(df['baseline value'], nbins=47)


Podemos ver a presença de alguns poucos outliers na classe 1 e 2. Já na classe 3 é observado um maior número de outliers.

In [ ]:
px.box(df['baseline value'], color = df['fetal_health'])

In [ ]:
px.histogram(df['accelerations'])

A distribuição dos dados acima decresce quando o valor do atributo aumenta, tornando o histograma assimétrico positivo

In [ ]:
px.histogram(df['fetal_movement'])


In [ ]:
px.histogram(df['uterine_contractions'])

In [ ]:
px.box(df['uterine_contractions'], color = df['fetal_health'])

In [ ]:
px.histogram(df['light_decelerations'])


In [ ]:
px.bar(df['severe_decelerations'].value_counts().sort_index(),
      color = df['severe_decelerations'].value_counts().sort_index().values,
       color_continuous_scale = 'RdBu'
      )

In [ ]:
px.histogram(df['prolongued_decelerations'])

In [ ]:
px.histogram(df['abnormal_short_term_variability'], nbins = 47)

In [ ]:
px.box(df['abnormal_short_term_variability'], color = df['fetal_health'])

In [ ]:
px.histogram(df['mean_value_of_short_term_variability'])

In [ ]:
px.box(df['mean_value_of_short_term_variability'], color = df['fetal_health'])

In [ ]:
px.histogram(df['percentage_of_time_with_abnormal_long_term_variability'])

In [ ]:
px.box(df['percentage_of_time_with_abnormal_long_term_variability'], color = df['fetal_health'])

In [ ]:
px.histogram(df['mean_value_of_long_term_variability'])

In [ ]:
px.box(df['mean_value_of_long_term_variability'], color = df['fetal_health'])

Abaixo está a parte de treinamento e classificação dos dados.Utilizei o método de padronização para realizar o pré-processamento e estratifiquei como foi pedido na tarefa. Além disso utilizei as principais métricas de avaliação dos algoritmos no qual o Random Forest se saiu superior aos demais.

In [ ]:
x = df.drop(['fetal_health'],axis = 1)
y = df['fetal_health']

x = StandardScaler().fit_transform(x)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,stratify =y,random_state = 42)

In [ ]:
clf = RandomForestClassifier()

clf.fit(x_train,y_train)

print('score:',clf.score(x_test,y_test))
print()
pred = clf.predict(x_test)
pred_proba = clf.predict_proba(x_test)

print(classification_report(y_test,pred))
print()
print('roc_auc:',roc_auc_score(y_test,pred_proba, multi_class = 'ovo'))
print()


In [ ]:
clf = LogisticRegression(max_iter = 1000)

clf.fit(x_train,y_train)

print('score:',clf.score(x_test,y_test))
print()
pred = clf.predict(x_test)
pred_proba = clf.predict_proba(x_test)

print(classification_report(y_test,pred))
print()
print('roc_auc:',roc_auc_score(y_test,pred_proba, multi_class = 'ovo'))

In [ ]:
clf = SVC(probability=True)

clf.fit(x_train,y_train)

print('score:',clf.score(x_test,y_test))
print()
pred = clf.predict(x_test)
pred_proba = clf.predict_proba(x_test)

print(classification_report(y_test,pred))
print()
print('roc_auc:',roc_auc_score(y_test,pred_proba, multi_class = 'ovo'))